In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

    
print("REPLICAS: ", strategy.num_replicas_in_sync)


In [ ]:

# set your directories
train_path = '/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/train/'
val_path = '/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/valid/'
test_path = '/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/test/'

# define a function to load images
def load_images_from_directory(dir_path, labels='inferred', label_mode='binary', image_size=(256, 256), batch_size=256):
    dataset = keras.utils.image_dataset_from_directory(
        directory=dir_path,
        labels=labels,
        label_mode=label_mode,
        image_size=image_size,
        batch_size=batch_size,
        shuffle=True
    )
    return dataset

# load your datasets
train_dataset = load_images_from_directory(train_path)
val_dataset = load_images_from_directory(val_path)
test_dataset = load_images_from_directory(test_path)

In [ ]:
def load_model_DenseNet121():
    model = keras.applications.DenseNet121(
        weights='imagenet',  # Load weights pre-trained on ImageNet.
        input_shape=(256, 256, 3),
        include_top=False
        )  # Do not include the ImageNet classifier at the top.
    return model

In [ ]:
def set_nontrainable_layers(model):
    model.trainable = False
    return model

In [ ]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    rescale_layer = layers.Rescaling(1./255, input_shape=(256, 256, 3))
    base_model = set_nontrainable_layers(model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(1, activation='sigmoid')
    
    
    model = models.Sequential([
        rescale_layer,
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
    return model

In [ ]:
def build_model():
    model = load_model_DenseNet121()
    model = add_last_layers(model)

    adam = optimizers.Adam(learning_rate=1e-4)

    model.compile(
        optimizer=adam,
        loss='binary_crossentropy',
        metrics=['accuracy', 'Precision', 'Recall']
    )
    return model

In [ ]:
def save_model(model, name):
    model.save(f"./model/{name}" + '.h5')

def load_model(name):
    return models.load_model(f"./model/{name}" + '.h5')


In [ ]:
def train_model(
        train_dataset,
        val_dataset,
        epochs=100,
        batch_size=32,
        patience=5
        ):
    es = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=patience,
        restore_best_weights=True,
        )
    with strategy.scope():
        model = build_model()
        history = model.fit(
            train_dataset,
            epochs=epochs, 
            batch_size=batch_size, 
            validation_data=val_dataset,
            callbacks=[es]
            )
    accuracy = round(history.history['val_accuracy'][-1], 2)
    date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    save_model(model, f"model_{accuracy}_{date}")
    return model, history

In [ ]:

model, history = train_model(train_dataset, val_dataset, epochs=1000, batch_size=512, patience=5)

In [ ]:
def plot_history(history):
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0, 1])
    plt.legend(loc='lower right')
    plt.show()

plot_history(history)

In [ ]:
accuracy = round(history.history['accuracy'][-1], 2)
date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
save_model(model, f"dnmodel_{accuracy}_{date}")

In [ ]:
res = model.evaluate(test_dataset)

In [ ]:
res